# bqplot Subscriber for Gießwagen

* class with traitlets

In [1]:
BROKER = "mqtt.eclipseprojects.io"
TOPIC = "gw01/duese03"

#BROKER = "broker.hivemq.com"
#TOPIC = "gw01/duese02"

# BROKER = "kleve.cool"
# TOPIC = "gw/rolf"
USER = "pub"
PW = "pub"

import paho.mqtt.client as mqtt
import numpy as np
#from numpy import zeros
import bqplot.pyplot as plt
from bqplot import LinearScale
from IPython.display import display, clear_output
import ipywidgets as widgets

In [2]:
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('HERE THE DECODED MQTT MESSAGES WILL BE SHOWN!\n')
display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [3]:
N = 20

x = np.arange(0,N)*5 #5mm resol.
t = z = np.zeros(N)

data = np.array([t,x,z]).T

tup = (0, 0, 0)

data

array([[ 0.,  0.,  0.],
       [ 0.,  5.,  0.],
       [ 0., 10.,  0.],
       [ 0., 15.,  0.],
       [ 0., 20.,  0.],
       [ 0., 25.,  0.],
       [ 0., 30.,  0.],
       [ 0., 35.,  0.],
       [ 0., 40.,  0.],
       [ 0., 45.,  0.],
       [ 0., 50.,  0.],
       [ 0., 55.,  0.],
       [ 0., 60.,  0.],
       [ 0., 65.,  0.],
       [ 0., 70.,  0.],
       [ 0., 75.,  0.],
       [ 0., 80.,  0.],
       [ 0., 85.,  0.],
       [ 0., 90.,  0.],
       [ 0., 95.,  0.]])

In [4]:
fig = plt.figure(title='Plant Profile Scanner')
x_scale = LinearScale(min=-10, max=1010)  # Set the x-axis limits
y_scale = LinearScale(min=0, max=300)  # Set the y-axis limits
profile = plt.scatter(x=data[:,1], y=data[:,2], colors=['blue'], marker='circle', default_size=100, default_opacities=[1.0], display_legend=False, scales={'x': x_scale, 'y': y_scale})
text_widget = plt.label(text = "0.00 s", x=[0.1], y=[0.1], colors=['black'])

In [5]:
display(fig)

Figure(axes=[Axis(scale=LinearScale(max=1010.0, min=-10.0)), Axis(orientation='vertical', scale=LinearScale(ma…

In [6]:

def on_message(client, userdata, message):
#    global payload
    global i, t, t0, x, z, traw
    global data, tup

    # Parse the received message
    payload = message.payload.decode('utf-8')
    (traw, x, z) = map(float, payload.strip('()').split(','))

    if (i==0): t0 = traw

    t = traw - t0
    
    tup = (t, x, z)
    
    s = f"{i:4d}  {t:12.2f}  {x:6.2f}  {z:6.2f}"

    i += 1

    data = np.roll(data, -1, axis = 0)
    data[-1,:] = tup

    with profile.hold_sync():
        profile.x = data[:,1]
        profile.y = data[:,2]

    # # Update the text widget with the latest 't' value
    text_widget.text = f"{t = :8.2f} s"


# MQTT setup
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    out.append_stdout("Connected with result code " + str(rc) + "\n")

client = mqtt.Client()

client.on_connect = on_connect
client.on_message = on_message

client.username_pw_set(USER, PW)
client.connect(BROKER, 1883, 60)

client.subscribe(TOPIC)

client.loop_start()

# Display the figure
display(fig)


Figure(axes=[Axis(scale=LinearScale(max=1010.0, min=-10.0)), Axis(orientation='vertical', scale=LinearScale(ma…

In [7]:
t

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [8]:
data

array([[ 0.,  0.,  0.],
       [ 0.,  5.,  0.],
       [ 0., 10.,  0.],
       [ 0., 15.,  0.],
       [ 0., 20.,  0.],
       [ 0., 25.,  0.],
       [ 0., 30.,  0.],
       [ 0., 35.,  0.],
       [ 0., 40.,  0.],
       [ 0., 45.,  0.],
       [ 0., 50.,  0.],
       [ 0., 55.,  0.],
       [ 0., 60.,  0.],
       [ 0., 65.,  0.],
       [ 0., 70.,  0.],
       [ 0., 75.,  0.],
       [ 0., 80.,  0.],
       [ 0., 85.,  0.],
       [ 0., 90.,  0.],
       [ 0., 95.,  0.]])

In [9]:
payload = "2.3, 1.0, 2.0"

In [10]:
(traw, x, z) = map(float, payload.strip('()').split(','))
tup = (traw, x, z)

In [11]:
tup

(2.3, 1.0, 2.0)